In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sympy as sp


In [74]:
x_1 = sp.Symbol('x_1')
x_2 = sp.Symbol('x_2')


sin(2x − y) − 1.2x = 0.4;
0.8x2 + 1.5y2 = 1 .

In [75]:
F = sp.Matrix([sp.sin(2*x_1-x_2) - 1.2*x_1 - 0.4, 0.8*x_1**2 + 1.5*x_2**2-1])
F

Matrix([
[-1.2*x_1 + sin(2*x_1 - x_2) - 0.4],
[      0.8*x_1**2 + 1.5*x_2**2 - 1]])

In [76]:
sp.lambdify([x_1, x_2], F)(1,2)

array([[-1.6],
       [ 5.8]])

In [77]:
dF_dx = F.jacobian([x_1,x_2])
sp.lambdify([x_1, x_2], dF_dx, 'numpy')(*np.array([1.0,2.3]))

array([[ 0.71067298, -0.95533649],
       [ 1.6       ,  6.9       ]])

In [9]:
F

Matrix([
[-1.2*x_1 + sin(2*x_1 - x_2) - 0.4],
[      0.8*x_1**2 + 1.5*x_2**2 - 1]])

In [143]:
def newton_solve(F: sp.Matrix, x0: np.array, tol, symbol_x, history_save = True):
    '''
    System of nonlinear equations solver (Newton method)
    
    Args:
        F (sp.Matrix): System of nonlinear equations F(x)=0
        x0 (np.array): first point in the iteration
        tol (float): tolerance
        symbol_x (list): list of symbol variables
    '''    
#     Create history array
    if history_save:
        history = [x0]
#     Define numerical F
    numerical_F = sp.lambdify(symbol_x, F)
#     Define symbol jacobian and numerical jacobian
    dF_dx = F.jacobian(symbol_x)
    numerical_dF_dx = sp.lambdify(symbol_x, dF_dx, 'numpy')
#     Solve the system of linear equations dF_dx(xk) *dxk = -F(xk) in a loop
    k=0
    while True:
        dx1 = np.linalg.solve(numerical_dF_dx(*x0), -numerical_F(*x0))
        dx1 = np.squeeze(dx1)
#         x1 = np.sum([x0, dx1], axis = 0)
        x1 = x0+dx1
#     Add value to history list
        if history_save:
            history.append(x1)
        if np.linalg.norm(x1 - x0)<=tol:
            print(f'k={k}: norm ||x_k+1 - x_k||<={tol}')
            return x1 if not history_save else x1, history
        k+=1
        x0 = x1

def merge_arrays(x_0, x_1):
    result = np.tile(x_1, (x_1.shape[0], 1))
    np.fill_diagonal(result, x_0)
    return result

def secant_solver(F: sp.Matrix, x0: np.array, x1: np.array, tol, symbol_x, history_save = True):
#     Create history array
    if history_save:
        history = [x0, x1]
        
#     Define numerical F
    numerical_F = sp.lambdify(symbol_x, F)
    
    k = 0
    while True:
        merged_matrix = merge_arrays(x0, x1)
        approx_jacobian = (np.squeeze(numerical_F(*merged_matrix.T)) - numerical_F(*x1))
        approx_jacobian/=(x0-x1)[:,None]
        dx2 = np.linalg.solve(approx_jacobian, -numerical_F(*x1))
        dx2 = np.squeeze(dx2)
        x2 = x1 + dx2
        
        if history_save:
            history.append(x2)
        if np.linalg.norm(x2 - x1)<=tol:
            print(f'k={k}: norm ||x_k+1 - x_k||<={tol}')
            return x2 if not history_save else x2, history
        k+=1
        x0 = x1
        x1 = x2
    

In [144]:
newton_x, newton_history = newton_solve(F, np.array([0.4, -0.75]), 0.0001, [x_1, x_2])
newton_history

k=3: norm ||x_k+1 - x_k||<=0.0001


[array([ 0.4 , -0.75]),
 array([ 0.5031025 , -0.73322862]),
 array([ 0.49141694, -0.73344703]),
 array([ 0.49123799, -0.7334613 ]),
 array([ 0.49123795, -0.7334613 ])]

In [145]:
x0 = np.array([0.4, -0.75])
dx = np.random.uniform(low =-0.1 ,high = 0.1, size = 2)
x1 = x0 + dx
# np.array([0.4, -0.75]), np.array([0.49123795, -0.7334613])
secant_x, secant_history = secant_solver(F,x0, x1, 0.0001, [x_1, x_2])
secant_x, secant_history

k=6: norm ||x_k+1 - x_k||<=0.0001


(array([ 0.49124177, -0.73345543]),
 [array([ 0.4 , -0.75]),
  array([ 0.43421855, -0.7445741 ]),
  array([ 0.49906948, -0.6306651 ]),
  array([ 0.481169 , -0.7333115]),
  array([ 0.49018675, -0.73669829]),
  array([ 0.49084864, -0.73445636]),
  array([ 0.49144606, -0.73353937]),
  array([ 0.49124614, -0.73343354]),
  array([ 0.49124177, -0.73345543])])